- Ziel: wie kann ich einen städtischen Datensatz mit Crowdsourced Zusatzinfos anreichern (OSM + WikiData)
- Jupyter Notebook mit Geopandas + Folium Karte
- Daten
    - OGD/GeoServer: Kunst im Stadtraum (KiöR und KuB)
    - Zeigen: Overpass Query + OSM Wiki (Wie suche ich nach passendes Tags?), taginfo
    - Join mit OSM: `tourism=artwork`
    - Join mit WikiData zu einzelnen Kunstwerken, ihren Künstlern + Foto vom Künstler und Kunstwerk
- OSM
    - OSM Wiki zeigen
    - Overpass Turbo zeigen + Export als GeoJSON
- WikiData
    - Was ist es?
    - Kurze Erklärung zum Datenmodell
    - Wie kann ich Geodaten aus WikiData beziehen?
    - SPARQL-Abfrage für KioR (Bild + Text + Link auf Wikipedia
    - Weitergehende Infos hier: https://www.wikidata.org/wiki/Wikidata:Training
- Ganzer Datensatz zum KioR joinen und dann als GPKG exportieren und z.B. in QGIS öffnen
- Vor/Nachteile, für welche Datensätze eignet sich dieses Vorgehen?
- Quellen und weiterführende Infos zu WikiData + OSM

In [ ]:
%pip install geopandas requests folium osm2geojson

In [1]:
import xml.etree.ElementTree as ET
import json

import requests
import folium
import osm2geojson
import geopandas
import pandas as pd

In [2]:
def get_layers_from_wfs(wfs_base_url):
    r = requests.get(wfs_base_url, params={
        'service': 'WFS',
        'version': '1.0.0',
        'request': 'GetCapabilities'
    })
    # XML parsen und die Layer-Informationen extrahieren
    root = ET.fromstring(r.content)
    namespaces = {
        'wfs': 'http://www.opengis.net/wfs'
    }
    layers = {}
    for feature_type in root.findall('wfs:FeatureTypeList/wfs:FeatureType', namespaces):
        layers[feature_type.find('wfs:Name', namespaces).text] = {
            'srs': feature_type.find('wfs:SRS', namespaces).text,
        }
    return layers

def overpass_query(q, endpoint='http://overpass.osm.ch/api/interpreter'):
    r = requests.get(endpoint, params={'data': q})
    r.raise_for_status()
    return r.json()

def style_layer(geojson, layer, **kwargs):
    # load GEOJSON, but don't add it to anything
    gj = folium.features.GeoJson(geojson)

    # iterate over GEOJSON features, pull out point coordinates, make Markers and add to layer
    for feature in gj.data['features']:
        if not feature or not feature['geometry']:
            continue
        if feature['geometry']['type'] == 'Point':
            tool_tip_text = '<pre>\n'
            properties = feature['properties']
            if properties.get('tags'):
                properties.update(properties['tags'])
                del properties['tags']
            for p in properties:
                tool_tip_text = tool_tip_text + str(p) + ': ' + str(feature['properties'][p]) + '\n'
            tool_tip_text = tool_tip_text + '\n</pre>'
            tool_tip = folium.Tooltip(tool_tip_text)
            folium.Marker(location=list(reversed(feature['geometry']['coordinates'])),
                          icon=folium.Icon(**kwargs),
                          tooltip=tool_tip
                          ).add_to(layer)

## GeoJSON für Kunst im öffentlichen Raum (KiöR)

[Via OGD-Portal](https://data.stadt-zuerich.ch/dataset?q=kunst+im+%C3%B6ffentlichen+raum) Datensatz [Kunst im Stadtraum](https://data.stadt-zuerich.ch/dataset/geo_kunst_im_stadtraum) von dort via GeoJSON auf das [Geoportal](https://www.stadt-zuerich.ch/geodaten/download/Kunst_im_Stadtraum?format=10009).

In [40]:
kioer_geojson_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Kunst_im_Stadtraum?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_kioer'
kioer_layer = 'view_kioer'
lv95 = 'EPSG:2056'
wgs84 = 'EPSG:4326'
wfs_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Kunst_im_Stadtraum' 
layers = get_layers_from_wfs(wfs_url)
layers

{'view_kioer': {'srs': 'EPSG:2056'}, 'view_kub': {'srs': 'EPSG:2056'}}

In [4]:
r = requests.get(wfs_url, params={
    'service': 'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typename': kioer_layer,
    'outputFormat': 'GeoJSON'
})
kioer_geo = r.json()
kioer_geo

{'type': 'FeatureCollection',
 'bbox': [8.474066, 47.327616, 8.605392, 47.425042],
 'features': [{'geometry': {'coordinates': [8.545688, 47.372467],
    'type': 'Point'},
   'id': 'view_kioer.84',
   'properties': {'autoren': 'Barbara Roth (*1950)\nThomas Ehrler (*1948)',
    'datierung': '1992',
    'id1': 84,
    'id_stadtplan': 1000084,
    'inventarnummer': '1396-01',
    'material': 'Bronze, Stein',
    'objectid': 84.0,
    'standort': 'Neumarkt, bei Neumarkt 4, AA4072',
    'titel': "L'étrangère"},
   'type': 'Feature'},
  {'geometry': None,
   'id': 'view_kioer.85',
   'properties': {'autoren': 'KünstlerIn nicht bekannt (nicht bekannt)\nAdolf Meyer (1867-1940)',
    'datierung': 'um 1550 (Figur um 1750, erneuert 1922)',
    'id1': 85,
    'id_stadtplan': 1000085,
    'inventarnummer': '1396-00',
    'material': 'Stein',
    'objectid': 85.0,
    'standort': None,
    'titel': '"Jupiterbrunnen"'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [8.552012, 47.35384], 'type'

In [5]:
# Basiskarte mit GeoJSON layer
m = folium.Map(location=[47.38, 8.53], zoom_start=13, tiles=None)
folium.raster_layers.WmsTileLayer(
    url='https://www.ogd.stadt-zuerich.ch/wms/geoportal/Basiskarte_Zuerich_Raster_Grau',
    layers='Basiskarte Zürich Raster Grau',
    name='Zürich - Basiskarte',
    fmt='image/png',
    overlay=False,
    control=False,
    autoZindex=False,
).add_to(m)
# KiöR-Daten hinzufügen
#folium.features.GeoJson(kioer_geo).add_to(m)
kioer_layer = folium.FeatureGroup(name='KiöR', show=True)
style_layer(kioer_geo, kioer_layer, icon_color='#031cff', icon='certificate', prefix='fa')
kioer_layer.add_to(m)
m

## OSM-Daten laden

Daten von OpenStreetMap (OSM) können via Overpass API geladen werden.
Overpass hat eine eigene Abfragesprache ([Overpass QL](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL)), mit der Objekte (Nodes, Ways, Relations) abgefragt werden können.

[Query in Overpass Turbo ausführen](https://osm.li/UfK)

In [6]:
artwork_zh = """
/*
Alle Kunstwerke (tourism=artwork) in der Stadt Zürich
*/
[out:json];
area["name"="Zürich"]["wikipedia"="de:Zürich"]->.perimeter; 
(
  nwr[tourism=artwork](area.perimeter);
);
out center;
"""
result = overpass_query(artwork_zh)
result

{'version': 0.6,
 'generator': 'Overpass API 0.7.56.8 7d656e78',
 'osm3s': {'timestamp_osm_base': '74131',
  'timestamp_areas_base': '74124',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 268472148,
   'lat': 47.3530162,
   'lon': 8.552309,
   'tags': {'artist_name': 'Jean Tinguely',
    'name': 'Heureka',
    'tourism': 'artwork',
    'wheelchair': 'yes',
    'wikidata': 'Q1378316'}},
  {'type': 'node',
   'id': 356298160,
   'lat': 47.3771287,
   'lon': 8.5399549,
   'tags': {'artist': 'Richard Kissling',
    'artist:wikidata': 'Q120459',
    'artist_name': 'Richard Kissling',
    'artwork_type': 'sculpture',
    'info': 'Hat die Credit Suisse gegründet und mit dem Gewinn den Gotthardtunnel gebaut',
    'name': 'Alfred Escher-Statue',
    'name:etymology:wikidata': 'Q115569',
    'subject:wikidata': 'Q115569',
    'tourism': 'artwork',
    'wheelchair': 'yes',
    'wi

In [7]:
osm_layer = folium.FeatureGroup(name='OSM: tourism=artwork', show=True)
result_geojson = osm2geojson.json2geojson(result)
style_layer(result_geojson, osm_layer, icon_color='#ff033e', icon='fire', prefix='fa')
osm_layer.add_to(m)

In [8]:
folium.LayerControl().add_to(m)
m

In [41]:
kioer_df = geopandas.GeoDataFrame.from_features(kioer_geo, crs={'init': wgs84})
kioer_df

/opt/tljh/user/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/tljh/user/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,geometry,autoren,datierung,id1,id_stadtplan,inventarnummer,material,objectid,standort,titel
0,POINT (8.54569 47.37247),Barbara Roth (*1950)\nThomas Ehrler (*1948),1992,84,1000084,1396-01,"Bronze, Stein",84.0,"Neumarkt, bei Neumarkt 4, AA4072",L'étrangère
1,None,KünstlerIn nicht bekannt (nicht bekannt)\nAdol...,"um 1550 (Figur um 1750, erneuert 1922)",85,1000085,1396-00,Stein,85.0,None,"""Jupiterbrunnen"""
2,POINT (8.55201 47.35384),Franz Wanger (1880-1945),1907 (platziert 1910),86,1000086,58-00,"Kupfer, Stein",86.0,"Zürichhorn, bei Bellerivestrasse 150, RI4672",Schweizerpsalm-Denkmal
3,POINT (8.52583 47.37414),KünstlerIn nicht bekannt (nicht bekannt),ca. 1916,87,1000087,85-00,"Bronze, Eisen",87.0,"Vorplatz Bezirksgebäude, bei Badenerstrasse 90...",[Kandelaber]
4,POINT (8.52136 47.37019),Silvio Mattioli (1929-2011),1984/1985,88,1000088,566-00,Chromnickelstahl,88.0,"Bankgebäude UBS Zürich-Wiedikon, Vorplatz , be...",[Stahlplastik]
...,...,...,...,...,...,...,...,...,...,...
401,POINT (8.53869 47.39200),L / B (*1972 / *1967),2002-2003,402,1000402,48-00,Beton,402.0,"Schaffhauserplatz, bei Schaffhauserstrasse 57 ...",Der Brunnen
402,POINT (8.54768 47.40317),Emil Schäfer (1878-1958)\nArnold Huggler (1894...,"1937 (Brunnen), 1941 (Plastik)",403,1000403,863-00,"Bronze (Plastik), Stein (Brunnen, Sockel)",403.0,"Berninaplatz, bei Schaffhauserstrasse 248, OE3799","[Brunnen mit ""Fuchs""]"
403,POINT (8.52653 47.37720),Uli Schoop (1903-1990),ca. 1975,404,1000404,563-00,"Bronze (Plastik), Beton (Sockel)",404.0,"vor Kollerhof, Kreisgebäude 4, bei Hohlstrasse...",[Fuchs]
404,POINT (8.54873 47.37053),Baptist Hoerbst (1850-1927),1883,405,1000405,19-00,Stein,405.0,"Heimplatz, bei Heimplatz 6, AA4138",Ignaz-Heim-Denkmal


In [42]:
osm_df = geopandas.GeoDataFrame.from_features(result_geojson, crs={'init': wgs84})
osm_df

/opt/tljh/user/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/tljh/user/lib/python3.7/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,geometry,type,id,artist_name,name,tourism,wheelchair,wikidata,artist,artist:wikidata,...,leisure,playground,backrest,access,mapillary,area,highway,lit,surface,year
0,POINT (8.55231 47.35302),node,268472148,Jean Tinguely,Heureka,artwork,yes,Q1378316,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (8.53995 47.37713),node,356298160,Richard Kissling,Alfred Escher-Statue,artwork,yes,Q27229673,Richard Kissling,Q120459,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POINT (8.54028 47.36588),node,489786064,NaN,Blumenuhr Bürkliplatz,artwork,yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POINT (8.54517 47.39695),node,692155260,NaN,Schwester Mond,artwork,limited,Q55166775,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POINT (8.53632 47.41425),node,693205583,NaN,Stier,artwork,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,POINT (8.53840 47.36974),way,617727049,NaN,Snow Moon,artwork,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,POINT (8.55001 47.39691),way,647033703,NaN,Georg Büchner Platz,artwork,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,yes,pedestrian,no,tartan,NaN
244,POINT (8.54121 47.36601),way,667718520,Hermann Hubacher,Ganymed,artwork,yes,Q27229968,NaN,Q870322,...,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN,NaN,1952
245,POINT (8.54027 47.36589),way,667718521,NaN,Blumenuhr Bürkliplatz,artwork,yes,NaN,NaN,NaN,...,garden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# drop all elements with empty geometry
kior_buf = kioer_df.dropna(subset=['geometry'])
osm_buf = osm_df.dropna(subset=['geometry'])

# Buffer um die Geometrien hinzufügen
kior_buf['geometry'] = kior_buf.geometry.buffer(0.00005)
osm_buf['geometry'] = osm_buf.geometry.buffer(0.00005)

/opt/tljh/user/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
# Basiskarte mit GeoJSON layer
bm = folium.Map(location=[47.38, 8.53], zoom_start=13, tiles=None)
folium.raster_layers.WmsTileLayer(
    url='https://www.ogd.stadt-zuerich.ch/wms/geoportal/Basiskarte_Zuerich_Raster_Grau',
    layers='Basiskarte Zürich Raster Grau',
    name='Zürich - Basiskarte',
    fmt='image/png',
    overlay=False,
    control=False,
    autoZindex=False,
).add_to(bm)
folium.features.GeoJson(kior_buf.to_json(), tooltip=folium.features.GeoJsonTooltip(
            fields=['titel', 'autoren', 'datierung', 'material'],
            aliases=['Titel:', 'Künstler:', 'Datierung:', 'Material:'],    
        )).add_to(bm)

style_function = lambda x: {'fillColor': '#FF0000', 'color': '#FF0000'}
folium.features.GeoJson(osm_buf.to_json(), style_function=style_function, tooltip=folium.features.GeoJsonTooltip(
            fields=['name', 'artist_name', 'wikidata'],
            aliases=['Titel:', 'Künstler:', "Wikidata:"],                      
        )).add_to(bm)
bm

In [43]:
#intersects
kior_buf.sindex.valid_query_predicates

{None,
 'contains',
 'contains_properly',
 'covers',
 'crosses',
 'intersects',
 'overlaps',
 'touches',
 'within'}

In [44]:
# spatial join über die beiden Geometrien
sjoin_kunst = geopandas.sjoin(kior_buf, osm_buf, how='left', predicate='intersects', lsuffix='kior', rsuffix='osm').reset_index()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(sjoin_kunst[['titel', 'name', 'autoren', 'artist_name', 'material_kior', 'material_osm', 'datierung']])

#sjoin_car_park = sjoin_car_park.rename(columns={"address": "address_parkendd", "adresse": "address_wfs"})
#sjoin_car_park[['name_wfs', 'name_parkendd', 'address_wfs', 'address_parkendd']]

,titel,name,autoren,artist_name,material_kior,material_osm,datierung
0,L'étrangère,NaN,Barbara Roth (*1950)\nThomas Ehrler (*1948),NaN,"Bronze, Stein",NaN,1992
1,Schweizerpsalm-Denkmal,NaN,Franz Wanger (1880-1945),NaN,"Kupfer, Stein",NaN,1907 (platziert 1910)
2,[Kandelaber],Kandelaber,KünstlerIn nicht bekannt (nicht bekannt),NaN,"Bronze, Eisen",NaN,ca. 1916
3,[Stahlplastik],NaN,Silvio Mattioli (1929-2011),NaN,Chromnickelstahl,NaN,1984/1985
4,"""Murmeltierbrunnen""",NaN,Alfred Schuhmacher (1883-1981),NaN,Stein,NaN,1937
5,[Wandbrunnen mit Broncekopf],NaN,Emil Abry (1901-1982),NaN,"Bronze (Wasserspeier), Kalkstein (Brunnen)",NaN,nicht bekannt
6,[Stehende weibliche Figur],Stehende weibliche Figur,Alfons Magg (1891-1967),Alfons Magg,"Bronze (Plastik), Beton (Sockel)",NaN,1936 (versetzt 1973)
7,[Fünf Poller],Fünf Poller,Barbara Roth (*1950)\nMartin Senn (*1960)\nFar...,Barbara Roth;Fariba Sepehrnia;Jan Morgenthaler...,"Eisen, angerostet",NaN,2009-2010
8,Muschel,Muschel,Annemie Fontana (1925-2002),Annemie Fontana,Polyester,NaN,1969-1972
9,[Büste],NaN,KünstlerIn nicht bekannt (nicht bekannt),NaN,"Marmor (Skulptur), Sandstein (Sockel)",NaN,nicht bekannt


In [28]:
sjoin_kunst.columns

Index(['index', 'geometry', 'autoren', 'datierung', 'id1', 'id_stadtplan',
       'inventarnummer', 'material_kior', 'objectid', 'standort', 'titel',
       'index_osm', 'type', 'id', 'artist_name', 'name', 'tourism',
       'wheelchair', 'wikidata', 'artist', 'artist:wikidata', 'artwork_type',
       'info', 'name:etymology:wikidata', 'subject:wikidata', 'wikipedia',
       'display', 'support', 'toilets:wheelchair', 'visibility', 'amenity',
       'bottle', 'dog', 'loc_name', 'start_date', 'addr:housenumber',
       'addr:street', 'website', 'official_name', 'uzh_landmark', 'note',
       'level', 'historic', 'image', 'drinking_water',
       'drinking_water:description', 'ref', 'fee', 'description',
       'material_osm', 'addr:full', 'location:description:de',
       'wikimedia_commons', 'height', 'survey:date', 'manufacturer',
       'description:en', 'inscription', 'leisure', 'playground', 'backrest',
       'access', 'mapillary', 'area', 'highway', 'lit', 'surface', 'year'],
   

In [45]:
# OSM-Einträge ohne Match
osm_no_match = osm_df[(~osm_df.id.isin(sjoin_kunst.id))].reset_index()
osm_no_match

,index,geometry,type,id,artist_name,name,tourism,wheelchair,wikidata,artist,...,leisure,playground,backrest,access,mapillary,area,highway,lit,surface,year
0,2,POINT (8.54028 47.36588),node,489786064,NaN,Blumenuhr Bürkliplatz,artwork,yes,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,POINT (8.54517 47.39695),node,692155260,NaN,Schwester Mond,artwork,limited,Q55166775,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,POINT (8.53632 47.41425),node,693205583,NaN,Stier,artwork,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,POINT (8.57885 47.36310),node,726662427,W. Martin,Elefant,artwork,NaN,Q27230070,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,POINT (8.55000 47.40736),node,942821382,NaN,NaN,artwork,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,240,POINT (8.54495 47.39678),node,8989236634,NaN,Bruder Sonne,artwork,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,241,POINT (8.54890 47.39786),node,8989556719,NaN,NaN,artwork,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,242,POINT (8.53840 47.36974),way,617727049,NaN,Snow Moon,artwork,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,243,POINT (8.55001 47.39691),way,647033703,NaN,Georg Büchner Platz,artwork,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,yes,pedestrian,no,tartan,NaN


In [46]:
osm_no_match.to_file("kunst_package.gpkg", layer='osm_no_match', driver="GPKG")
sjoin_kunst.to_file("kunst_package.gpkg", layer='kioer_osm_match', driver="GPKG")

In [47]:
osm_no_match.crs

<Geographic 2D CRS: +init=epsg:4326 +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich